
# сравнение функций потерь от сжатого и сырого контекстов.

Основная цель оптимизации — обучить отдельную сеть или модуль, который будет динамически сжимать или изменять контекст так, чтобы предсказания исходной языковой модели оставались точными.

### Упрощенная Архитектура для Оптимизации Контекста

#### Цель
Обучить модуль сжатия контекста, который будет оптимизировать входной контекст для существующей языковой модели, минимизируя объем и избыточность данных, но сохраняя все необходимые элементы для точных предсказаний.

### Структура и Процесс Обучения

1. **Имеющаяся Языковая Модель:**
   - Пусть \( M \) — это наша основная языковая модель, которая предсказывает следующие слова или предложения на основе текущего фрагмента и контекста.
   - Модель \( M \) обладает хорошей точностью предсказаний при использовании полной предыстории.

2. **Модуль Оптимизации Контекста:**
   - Добавляем модуль \( O \), который принимает на вход предысторию \( C \) и текущий фрагмент \( S \) и выдает сжатый или оптимизированный контекст \( C' \).
   - Модуль \( O \) может быть реализован как легковесная нейронная сеть, например, на основе рекуррентных сетей или слоев внимания, которая фокусируется на выделении ключевых частей контекста.

3. **Обучение Модуля Оптимизации:**
   - **Вход:** Предыстория \( C \) и текущий фрагмент \( S \).
   - **Выход:** Сжатый контекст \( C' = O(C, S) \).
   - **Целевая Функция:** Оптимизировать \( C' \) так, чтобы предсказания модели \( M \) на основе \( C' \) и \( S \) оставались максимально близкими к предсказаниям на основе полной предыстории \( C \).
   - Используем функцию потерь:
     \[
     L(\hat{Y}_{C'}, \hat{Y}_{C}) = \text{Loss}(M(C', S), M(C, S))
     \]
   - Здесь \(\hat{Y}_{C'}\) и \(\hat{Y}_{C}\) — предсказания модели \( M \) на основе сжатого и полного контекста соответственно.

4. **Процесс Обучения:**
   - **Шаг 1:** Для каждой последовательности данных создается пара: полный контекст и текущий фрагмент.
   - **Шаг 2:** Модуль \( O \) сжимает контекст, производя \( C' \).
   - **Шаг 3:** Языковая модель \( M \) делает предсказания на основе \( C' \) и сравнивается с предсказаниями на основе \( C \).
   - **Шаг 4:** Функция потерь минимизируется для обучения модуля \( O \) таким образом, чтобы сжатие контекста минимально влияло на точность предсказаний.

### Ожидаемый Результат

- Модуль \( O \) должен научиться выделять только те части контекста, которые действительно необходимы для точных предсказаний модели \( M \).
- Сжатый контекст \( C' \) будет специфичен для данной языковой модели и адаптирован для минимизации избыточности без потери точности.

## Модульная структура

In [11]:
# Вспомогательные классы и методы

import torch
from transformers import GPT2Tokenizer
import torch.nn as nn
import plotly.graph_objects as go
from IPython.display import display
import os


# Подготовка данных
def prepare_data(example, max_length=50):
    text = example["text"].strip()
    if len(text) == 0 or "." not in text:
        return None, None

    split_point = text.find(".") + 1
    if split_point >= len(text):
        return None, None

    context = text[:split_point].strip()  # Предыстория до первой точки
    current_fragment = text[split_point : split_point + max_length].strip()  # Текущий фрагмент

    if len(context) == 0 or len(current_fragment) == 0:
        return None, None

    return context, current_fragment


# Функции для сохранения и загрузки модели
def save_model(model, optimizer, path="context_optimizer.pth"):
    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        path,
    )
    print(f"Model saved to {path}")


def load_model(model, optimizer, path="context_optimizer.pth"):
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        print(f"Model loaded from {path}")
    else:
        print(f"Model file not found at {path}")


# Визуализация с использованием Plotly FigureWidget
def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Raw Context Loss"))
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Compressed Context Loss"))
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Fragment Only Loss"))
    fig.update_layout(title="Графики потерь во время обучения", xaxis_title="Эпоха", yaxis_title="Среднее значение потерь", template="plotly_dark")
    display(fig)
    return fig


def update_loss_plot(fig, epoch, raw_loss, compressed_loss, fragment_loss):
    fig.data[0].x += (epoch,)
    fig.data[0].y += (raw_loss,)
    fig.data[1].x += (epoch,)
    fig.data[1].y += (compressed_loss,)
    fig.data[2].x += (epoch,)
    fig.data[2].y += (fragment_loss,)
    fig.show()


# Функция для расчета compression ratio
def calculate_compression_ratio(raw_embedding, compressed_embedding):
    raw_norm = torch.norm(raw_embedding, p=2, dim=1)
    compressed_norm = torch.norm(compressed_embedding, p=2, dim=1)
    return compressed_norm / raw_norm


# Функция для расчета compression ratio
def calculate_compression_ratio(raw_embedding, compressed_embedding):
    raw_norm = torch.norm(raw_embedding, p=2, dim=1)
    compressed_norm = torch.norm(compressed_embedding, p=2, dim=1)
    return compressed_norm / raw_norm


# Предсказание на основе сжатого контекста
def predict_with_compressed_context(compressed_context, current_fragment, tokenizer, gpt2_model, device):
    inputs = tokenizer(current_fragment, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = gpt2_model(input_ids=input_ids, output_hidden_states=True)
        logits = outputs.logits

    predicted_index = torch.argmax(logits[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])
    return predicted_token


# Предсказание на основе сырого контекста
def predict_with_raw_context(raw_context_embedding, current_fragment, tokenizer, gpt2_model, device):
    inputs = tokenizer(current_fragment, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = gpt2_model(input_ids=input_ids, output_hidden_states=True)
        logits = outputs.logits

    predicted_index = torch.argmax(logits[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])
    return predicted_token


# Предсказание только по текущему фрагменту
def predict_with_fragment(current_fragment, tokenizer, gpt2_model, device):
    inputs = tokenizer(current_fragment, return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = gpt2_model(input_ids=input_ids, output_hidden_states=True)
        logits = outputs.logits

    predicted_index = torch.argmax(logits[0, -1, :]).item()
    predicted_token = tokenizer.decode([predicted_index])
    return predicted_token

In [12]:
# Модель сжатия контекста


class ContextOptimizer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContextOptimizer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, input_size)
        self.relu = nn.ReLU()

    def forward(self, context):
        x = self.relu(self.fc1(context))
        x = self.fc2(x)
        return x


# Функция для сжатия контекста
def compress_context(context, model, tokenizer, gpt2_model, device):
    inputs = tokenizer(context, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = gpt2_model(**inputs, output_hidden_states=True)
        context_embedding = outputs.hidden_states[-1].mean(dim=1)

    compressed_context = model(context_embedding)
    return compressed_context, context_embedding

In [19]:
import time


def train_context_optimizer(optimizer, dataset, tokenizer, gpt2_model, epochs=10, device="cpu"):
    criterion = nn.MSELoss()
    opt = optim.Adam(optimizer.parameters(), lr=0.001)
    raw_loss_values = []
    compressed_loss_values = []
    fragment_loss_values = []
    compression_ratios = []

    # Инициализация графика потерь
    loss_fig = create_loss_plot()

    for epoch in range(epochs):
        total_raw_loss = 0
        total_compressed_loss = 0
        total_fragment_loss = 0
        total_compression_ratio = 0
        count = 0

        examples = []  # Список для сохранения примеров для диагностики
        start_time = time.time()
        time_measurements = []

        for i in range(len(dataset)):
            context, current_fragment = prepare_data(dataset[i])
            if context is None or current_fragment is None:
                continue
            cycle_start_time = time.time()
            compressed_context, raw_context_embedding = compress_context(context, optimizer, tokenizer, gpt2_model, device)

            # Подготовка целевого предсказания
            target_input = tokenizer(current_fragment + " ", return_tensors="pt").to(device)
            with torch.no_grad():
                target_output = gpt2_model(**target_input, output_hidden_states=True)
                target_embedding = target_output.hidden_states[-1].mean(dim=1)

            # Вычисление потерь для сжатого контекста в комбинации с текущим фрагментом
            compressed_inputs = tokenizer(current_fragment, return_tensors="pt").to(device)
            compressed_inputs_embeds = compressed_context + gpt2_model.transformer.wte(compressed_inputs["input_ids"])
            # Исправление: включаем output_hidden_states=True
            compressed_outputs = gpt2_model(inputs_embeds=compressed_inputs_embeds, output_hidden_states=True).hidden_states[-1].mean(dim=1)
            compressed_loss = criterion(compressed_outputs, target_embedding)

            # Вычисление потерь для полного контекста в комбинации с текущим фрагментом
            raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding)

            # Вычисление потерь для предсказания только по текущему фрагменту
            fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
            with torch.no_grad():
                fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
                fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding)

            # Вычисление compression ratio
            compression_ratio = calculate_compression_ratio(raw_context_embedding, compressed_context)
            total_compression_ratio += compression_ratio.item()

            # Обновление оптимизатора только по сжатым потерям
            opt.zero_grad()
            compressed_loss.backward()
            opt.step()

            total_raw_loss += raw_loss.item()
            total_compressed_loss += compressed_loss.item()
            total_fragment_loss += fragment_loss.item()
            count += 1
            if i == 4:  # После 5-го цикла оцениваем время эпохи
                avg_time_per_cycle = sum(time_measurements) / len(time_measurements)
                time_per_epoch_estimation = avg_time_per_cycle * len(dataset)
                print(f"Estimated time for epoch: {time_per_epoch_estimation:.2f} seconds")

            # Сохранение примеров для диагностики
            if len(examples) < 3:  # Сохраняем три примера для вывода
                with torch.no_grad():
                    compressed_pred = predict_with_compressed_context(compressed_context, current_fragment, tokenizer, gpt2_model, device)
                    raw_pred = predict_with_raw_context(raw_context_embedding, current_fragment, tokenizer, gpt2_model, device)
                    fragment_pred = predict_with_fragment(current_fragment, tokenizer, gpt2_model, device)

                # Проверяем и обрабатываем compressed_context корректно перед декодированием
                if compressed_context is not None and isinstance(compressed_context, torch.Tensor):
                    # Пытаемся преобразовать в список токенов, исключая None
                    compressed_tokens = [token for token in compressed_context.squeeze().tolist() if isinstance(token, int)]
                    compressed_context_text = tokenizer.decode(compressed_tokens) if compressed_tokens else "N/A"
                else:
                    compressed_context_text = "N/A"

                examples.append(
                    {
                        "context": context,
                        "compressed_context": compressed_context_text,
                        "current_fragment": current_fragment,
                        "compressed_prediction": compressed_pred,
                        "raw_prediction": raw_pred,
                        "fragment_prediction": fragment_pred,
                        "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    }
                )

        avg_raw_loss = total_raw_loss / count if count > 0 else 0
        avg_compressed_loss = total_compressed_loss / count if count > 0 else 0
        avg_fragment_loss = total_fragment_loss / count if count > 0 else 0
        avg_compression_ratio = total_compression_ratio / count if count > 0 else 0
        raw_loss_values.append(avg_raw_loss)
        compressed_loss_values.append(avg_compressed_loss)
        fragment_loss_values.append(avg_fragment_loss)

        print(
            f"Epoch {epoch+1}, Raw Context Loss: {avg_raw_loss:.4f}, Compressed Context Loss: {avg_compressed_loss:.4f}, Fragment Loss: {avg_fragment_loss:.4f}, Avg Compression Ratio: {avg_compression_ratio:.4f}"
        )

        # Обновление графика после каждой эпохи
        update_loss_plot(loss_fig, epoch + 1, avg_raw_loss, avg_compressed_loss, avg_fragment_loss)

        # Вывод примеров для диагностики
        print(f"\n=== Диагностика после эпохи {epoch+1} ===")
        for idx, example in enumerate(examples):
            print(f"\nПример {idx+1}:")
            print(f"Предыстория: {example['context']}")
            print(f"Сжатый контекст: {example['compressed_context']}")
            print(f"Текущий фрагмент: {example['current_fragment']}")
            print(f"Предсказание по сжатому контексту: {example['compressed_prediction']}")
            print(f"Предсказание по сырому контексту: {example['raw_prediction']}")
            print(f"Предсказание по текущему фрагменту: {example['fragment_prediction']}")
            print(f"Целевой текст: {example['target']}")
    return raw_loss_values, compressed_loss_values, fragment_loss_values

In [20]:
# Инициализация и запуск обучения

import torch.optim as optim
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Настройки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка модели GPT-2 и токенизатора
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Загрузка датасета
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Инициализация модели сжатия контекста
input_size = 768
hidden_size = 256
context_optimizer = ContextOptimizer(input_size, hidden_size).to(device)

# Загрузка сохраненной модели, если есть
load_model(context_optimizer, context_optimizer, "context_optimizer.pth")

# Запуск обучения
raw_loss_values, compressed_loss_values, fragment_loss_values = train_context_optimizer(context_optimizer, dataset, tokenizer, gpt2_model, epochs=10, device=device)

# Сохранение модели после обучения
save_model(context_optimizer, context_optimizer, "context_optimizer.pth")

Using device: cpu
Model file not found at context_optimizer.pth


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Raw Context Loss',
              'type': 'scatter',
              'uid': '94ad05bb-3a0e-464f-9996-1140a2ce9dab',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'name': 'Compressed Context Loss',
              'type': 'scatter',
              'uid': '2368fe70-416f-4651-98fc-cd70ad6384ef',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'name': 'Fragment Only Loss',
              'type': 'scatter',
              'uid': '2a8e436d-b960-423f-8b24-c8ed998288bf',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Графики потерь во время обучения'},
               'xaxis': {'title': {'text': 'Эпоха'}},
               'yaxis': {'title': {'text': 'Среднее значение потерь'}}}
})

## Слитная архитектура

In [5]:
# Импорт необходимых модулей
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch import nn, optim
from datasets import load_dataset
import time
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Проверка доступности устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка модели GPT-2 и токенизатора
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Перевод модели в режим оценки
model.eval()


# Определение модуля оптимизации контекста
class ContextOptimizer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContextOptimizer, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, input_size)
        self.relu = nn.ReLU()

    def forward(self, context):
        x = self.relu(self.fc1(context))
        x = self.fc2(x)
        return x


# Параметры оптимизатора
input_size = 768  # Размер скрытого состояния модели GPT-2
hidden_size = 256
optimizer = ContextOptimizer(input_size, hidden_size).to(device)

# Загрузка датасета "wikitext"
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")


# Функция для подготовки данных: извлечение контекста и текущего фрагмента
def prepare_data(example, max_length=50):
    text = example["text"].strip()
    if len(text) == 0 or "." not in text:
        return None, None

    # Разделение текста на предысторию и текущий фрагмент
    split_point = text.find(".") + 1
    if split_point >= len(text):
        return None, None

    context = text[:split_point].strip()  # Предыстория до первой точки
    current_fragment = text[split_point : split_point + max_length].strip()  # Текущий фрагмент

    if len(context) == 0 or len(current_fragment) == 0:
        return None, None

    return context, current_fragment


# Функция для сжатия контекста
def compress_context(context, optimizer):
    # Получение эмбеддингов контекста из токенов
    inputs = tokenizer(context, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        context_embedding = outputs.hidden_states[-1].mean(dim=1)  # Среднее по последнему слою

    # Сжатие контекста через оптимизатор
    compressed_context = optimizer(context_embedding)
    return compressed_context


# Функция потерь и обучение с обновлением графика после каждой эпохи
def train_context_optimizer(optimizer, dataset, epochs=10):
    criterion = nn.MSELoss()  # Можно использовать другие функции потерь
    opt = optim.Adam(optimizer.parameters(), lr=0.001)
    raw_loss_values = []  # Список для хранения значений потерь с сырым контекстом
    compressed_loss_values = []  # Список для хранения значений потерь с сжатым контекстом

    # Создание интерактивного графика
    fig = go.FigureWidget()
    fig.add_scatter(x=[], y=[], mode="lines+markers", name="Raw Context Loss", line=dict(color="blue"))
    fig.add_scatter(x=[], y=[], mode="lines+markers", name="Compressed Context Loss", line=dict(color="red"))
    fig.update_layout(title="Графики потерь во время обучения", xaxis_title="Эпоха", yaxis_title="Среднее значение потерь", template="plotly_dark")
    display(fig)

    for epoch in range(epochs):
        total_raw_loss = 0
        total_compressed_loss = 0
        count = 0
        start_time = time.time()

        for i in range(len(dataset)):
            context, current_fragment = prepare_data(dataset[i])
            if context is None or current_fragment is None:
                continue

            # Сжатие контекста
            compressed_context = compress_context(context, optimizer)

            # Получение эмбеддингов для сырого контекста
            inputs = tokenizer(context, return_tensors="pt").to(device)
            with torch.no_grad():
                raw_outputs = model(**inputs, output_hidden_states=True)
                raw_context_embedding = raw_outputs.hidden_states[-1].mean(dim=1)  # Среднее по последнему слою

            # Подготовка целевого предсказания для функции потерь
            target_input = tokenizer(current_fragment + " ", return_tensors="pt").to(device)
            with torch.no_grad():
                target_output = model(**target_input, output_hidden_states=True)
                target_embedding = target_output.hidden_states[-1].mean(dim=1)

            # Вычисление потерь для сырого контекста
            raw_loss = criterion(raw_context_embedding, target_embedding)

            # Вычисление потерь для сжатого контекста
            compressed_loss = criterion(compressed_context, target_embedding)

            # Обновление оптимизатора только по сжатым потерям
            opt.zero_grad()
            compressed_loss.backward()
            opt.step()

            total_raw_loss += raw_loss.item()
            total_compressed_loss += compressed_loss.item()
            count += 1

        avg_raw_loss = total_raw_loss / count if count > 0 else 0
        avg_compressed_loss = total_compressed_loss / count if count > 0 else 0
        raw_loss_values.append(avg_raw_loss)  # Сохранение среднего значения потерь для сырого контекста
        compressed_loss_values.append(avg_compressed_loss)  # Сохранение среднего значения потерь для сжатого контекста
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - start_time
        print(f"Epoch {epoch+1}, Raw Context Loss: {avg_raw_loss:.4f}, Compressed Context Loss: {avg_compressed_loss:.4f}, Epoch Time: {epoch_time / 60:.2f} minutes")

        # Обновление графика после каждой эпохи
        with fig.batch_update():
            fig.data[0].x = list(range(1, len(raw_loss_values) + 1))
            fig.data[0].y = raw_loss_values
            fig.data[1].x = list(range(1, len(compressed_loss_values) + 1))
            fig.data[1].y = compressed_loss_values

    return raw_loss_values, compressed_loss_values


# Запуск обучения и сохранение значений потерь
raw_loss_values, compressed_loss_values = train_context_optimizer(optimizer, dataset)

# Сохранение модели
save_model(optimizer, optimizer)

Using device: cpu


FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines+markers',
              'name': 'Raw Context Loss',
              'type': 'scatter',
              'uid': '78a544dd-cba2-4c31-81fb-0e62f1522d6b',
              'x': [],
              'y': []},
             {'line': {'color': 'red'},
              'mode': 'lines+markers',
              'name': 'Compressed Context Loss',
              'type': 'scatter',
              'uid': 'f01167cd-c8ea-422e-848c-0181ec70fcb2',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Графики потерь во время обучения'},
               'xaxis': {'title': {'text': 'Эпоха'}},
               'yaxis': {'title': {'text': 'Среднее значение потерь'}}}
})

Epoch 1, Raw Context Loss: 1.6017, Compressed Context Loss: 0.9392, Epoch Time: 37.61 minutes
Epoch 2, Raw Context Loss: 1.6017, Compressed Context Loss: 0.7524, Epoch Time: 37.78 minutes


KeyboardInterrupt: 